In [1]:
!pip install nba_api


# Create Master Scoring

In [10]:
import numpy as np
import pandas as pd

In [11]:
start = 2014
end = 2022

In [12]:
# import necessary modules
from nba_api.stats.endpoints import leaguedashplayerptshot

# set up empty dataframe to store data
scoring = pd.DataFrame()

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):

    # set up parameters for API request
    year = str(season) + "-" + str(season + 1)[-2:]
    season_type = "Regular Season"
    per_mode = "PerGame"
    
    # make API request
    response = leaguedashplayerptshot.LeagueDashPlayerPtShot(
        season=year,
        season_type_all_star=season_type,
        per_mode_simple=per_mode,
    )
    
    # get dataframe from response and append to overall dataframe
    df = response.get_data_frames()[0]
    df = df.drop(['PLAYER_LAST_TEAM_ID', 'PLAYER_LAST_TEAM_ABBREVIATION', 'AGE', 'G'], axis=1)
    df['SEASON'] = year  # add new column for season
    scoring = scoring.append(df)

# reset index of dataframe
scoring = scoring.reset_index(drop=True)


In [13]:
scoring.head()

,PLAYER_ID,PLAYER_NAME,GP,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FG2A_FREQUENCY,FG2M,FG2A,FG2_PCT,FG3A_FREQUENCY,FG3M,FG3A,FG3_PCT,SEASON
0,201566,Russell Westbrook,67,1.0,9.33,20.60,0.453,0.484,0.793,8.06,16.33,0.494,0.207,1.27,4.27,0.297,2014-15
1,977,Kobe Bryant,35,1.0,7.60,19.60,0.388,0.427,0.732,6.06,14.34,0.422,0.268,1.54,5.26,0.293,2014-15
2,200746,LaMarcus Aldridge,71,1.0,9.28,19.41,0.478,0.492,0.925,8.76,17.94,0.488,0.075,0.52,1.46,0.356,2014-15
3,2546,Carmelo Anthony,39,1.0,8.97,19.41,0.462,0.502,0.768,7.44,14.90,0.499,0.232,1.54,4.51,0.341,2014-15
4,2544,LeBron James,68,1.0,9.04,17.87,0.506,0.555,0.728,7.31,13.00,0.562,0.272,1.74,4.87,0.356,2014-15


In [14]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    shooting_efficiency = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Efficiency',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    shooting_efficiency['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(shooting_efficiency)

# concatenate all the dataframes into a single dataframe
shooting_eff = pd.concat(season_dfs)
shooting_eff.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,MIN,POINTS,DRIVE_PTS,...,PULL_UP_PTS,PULL_UP_FG_PCT,PAINT_TOUCH_PTS,PAINT_TOUCH_FG_PCT,POST_TOUCH_PTS,POST_TOUCH_FG_PCT,ELBOW_TOUCH_PTS,ELBOW_TOUCH_FG_PCT,EFF_FG_PCT,SEASON
0,201985,AJ Price,1610612739,CLE,26,11,15,12.4,5.1,1.6,...,2.8,0.405,0.2,0.500,0.1,1.000,0.1,0.500,0.427,2014-15
1,201166,Aaron Brooks,1610612741,CHI,82,50,32,23.0,11.6,4.9,...,3.5,0.386,0.1,0.294,0.0,0.000,0.2,0.455,0.495,2014-15
2,203932,Aaron Gordon,1610612753,ORL,47,14,33,17.0,5.2,1.2,...,0.3,0.286,2.0,0.695,0.1,0.333,0.4,0.667,0.478,2014-15
3,203940,Adreian Payne,1610612750,MIN,32,7,25,23.1,6.7,1.0,...,0.6,0.417,2.1,0.500,0.7,0.323,1.1,0.333,0.416,2014-15
4,201143,Al Horford,1610612737,ATL,76,56,20,30.5,15.2,1.3,...,0.4,0.400,4.8,0.669,2.1,0.456,2.3,0.476,0.544,2014-15


In [15]:
shooting_eff = shooting_eff.drop(['TEAM_ID', 'GP', 'W', 'L', 'POINTS'], axis=1)

In [16]:
shooting_eff.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,MIN,DRIVE_PTS,DRIVE_FG_PCT,CATCH_SHOOT_PTS,CATCH_SHOOT_FG_PCT,PULL_UP_PTS,PULL_UP_FG_PCT,PAINT_TOUCH_PTS,PAINT_TOUCH_FG_PCT,POST_TOUCH_PTS,POST_TOUCH_FG_PCT,ELBOW_TOUCH_PTS,ELBOW_TOUCH_FG_PCT,EFF_FG_PCT,SEASON
0,201985,AJ Price,CLE,12.4,1.6,0.486,1.2,0.323,2.8,0.405,0.2,0.500,0.1,1.000,0.1,0.500,0.427,2014-15
1,201166,Aaron Brooks,CHI,23.0,4.9,0.465,2.6,0.423,3.5,0.386,0.1,0.294,0.0,0.000,0.2,0.455,0.495,2014-15
2,203932,Aaron Gordon,ORL,17.0,1.2,0.449,0.9,0.246,0.3,0.286,2.0,0.695,0.1,0.333,0.4,0.667,0.478,2014-15
3,203940,Adreian Payne,MIN,23.1,1.0,0.361,2.0,0.369,0.6,0.417,2.1,0.500,0.7,0.323,1.1,0.333,0.416,2014-15
4,201143,Al Horford,ATL,30.5,1.3,0.567,5.7,0.471,0.4,0.400,4.8,0.669,2.1,0.456,2.3,0.476,0.544,2014-15


In [17]:
from nba_api.stats.endpoints import leaguedashplayerstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player scoring data for the season
    scoring_data = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
    ).get_data_frames()[0]
    scoring_data['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(scoring_data)

# concatenate all the dataframes into a single dataframe
gen_trad = pd.concat(season_dfs)

In [18]:
gen_trad = gen_trad[['PLAYER_ID', 'PLAYER_NAME', 'DD2', 'TD3', 'SEASON']]
gen_trad.head()

,PLAYER_ID,PLAYER_NAME,DD2,TD3,SEASON
0,201985,AJ Price,0,0,2014-15
1,201166,Aaron Brooks,1,0,2014-15
2,203932,Aaron Gordon,1,0,2014-15
3,203940,Adreian Payne,3,0,2014-15
4,201143,Al Horford,14,1,2014-15


In [19]:
from nba_api.stats.endpoints import leaguedashplayerstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player scoring data for the season
    scoring_data = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
        measure_type_detailed_defense='Scoring',
        per_mode_detailed = 'PerGame'
    ).get_data_frames()[0]
    scoring_data['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(scoring_data)

# concatenate all the dataframes into a single dataframe
gen_score = pd.concat(season_dfs)
gen_score = gen_score[[ 'PLAYER_ID', 'PLAYER_NAME', 'PCT_PTS_FB', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT', 'PCT_PTS_2PT_MR', 'SEASON']]
gen_score.head()


,PLAYER_ID,PLAYER_NAME,PCT_PTS_FB,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_PTS_2PT_MR,SEASON
0,201985,AJ Price,0.038,0.211,0.226,0.316,2014-15
1,201166,Aaron Brooks,0.095,0.071,0.363,0.105,2014-15
2,203932,Aaron Gordon,0.210,0.251,0.617,0.041,2014-15
3,203940,Adreian Payne,0.028,0.136,0.535,0.310,2014-15
4,201143,Al Horford,0.082,0.124,0.522,0.356,2014-15


In [20]:
from nba_api.stats.endpoints import leaguedashplayerstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player advanced stats data for the season
    adv_stats = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
        measure_type_detailed_defense='Advanced',
    ).get_data_frames()[0]
    # add season column
    adv_stats['SEASON'] = season_str
    # append the dataframe to the list
    season_dfs.append(adv_stats)

# concatenate all the dataframes into a single dataframe
gen_adv = pd.concat(season_dfs)
gen_adv.head()


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,E_PACE_RANK,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,SEASON
0,201985,AJ Price,AJ,1610612739,CLE,28.0,26,11,15,0.423,...,476,453,453,238,374,369,313,278,408,2014-15
1,201166,Aaron Brooks,Aaron,1610612741,CHI,30.0,82,50,32,0.610,...,371,388,388,226,82,74,125,112,279,2014-15
2,203932,Aaron Gordon,Aaron,1610612753,ORL,19.0,47,14,33,0.298,...,206,214,214,312,313,324,311,330,191,2014-15
3,203940,Adreian Payne,Adreian,1610612750,MIN,24.0,32,7,25,0.219,...,386,371,371,433,317,315,232,209,303,2014-15
4,201143,Al Horford,Al,1610612737,ATL,29.0,76,56,20,0.737,...,264,251,251,26,17,42,25,57,45,2014-15


In [21]:
gen_adv.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RATING', 'OFF_RATING',
       'sp_work_OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'sp_work_DEF_RATING', 'E_NET_RATING', 'NET_RATING',
       'sp_work_NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT',
       'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'E_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'sp_work_PACE',
       'PIE', 'POSS', 'FGM', 'FGA', 'FGM_PG', 'FGA_PG', 'FG_PCT', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'E_OFF_RATING_RANK',
       'OFF_RATING_RANK', 'sp_work_OFF_RATING_RANK', 'E_DEF_RATING_RANK',
       'DEF_RATING_RANK', 'sp_work_DEF_RATING_RANK', 'E_NET_RATING_RANK',
       'NET_RATING_RANK', 'sp_work_NET_RATING_RANK', 'AST_PCT_RANK',
       'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK',
       'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'E_TOV_PCT_

In [22]:
gen_adv = gen_adv[['PLAYER_ID', 'PLAYER_NAME', 'OFF_RATING', 'USG_PCT', 'SEASON']]

In [23]:
gen_adv.head()

,PLAYER_ID,PLAYER_NAME,OFF_RATING,USG_PCT,SEASON
0,201985,AJ Price,94.4,0.218,2014-15
1,201166,Aaron Brooks,107.2,0.246,2014-15
2,203932,Aaron Gordon,97.9,0.151,2014-15
3,203940,Adreian Payne,98.8,0.173,2014-15
4,201143,Al Horford,107.5,0.220,2014-15


In [24]:
master_scoring = pd.merge(scoring, gen_adv, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='left')
master_scoring = pd.merge(master_scoring, gen_score, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='left')
master_scoring = pd.merge(master_scoring, gen_trad, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='left')
master_scoring = pd.merge(master_scoring, shooting_eff, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='left')

master_scoring.head()

,PLAYER_ID,PLAYER_NAME,GP,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FG2A_FREQUENCY,FG2M,...,CATCH_SHOOT_FG_PCT,PULL_UP_PTS,PULL_UP_FG_PCT,PAINT_TOUCH_PTS,PAINT_TOUCH_FG_PCT,POST_TOUCH_PTS,POST_TOUCH_FG_PCT,ELBOW_TOUCH_PTS,ELBOW_TOUCH_FG_PCT,EFF_FG_PCT
0,201566,Russell Westbrook,67,1.0,9.33,20.60,0.453,0.484,0.793,8.06,...,0.361,8.6,0.359,1.5,0.487,0.8,0.463,0.7,0.621,0.455
1,977,Kobe Bryant,35,1.0,7.60,19.60,0.388,0.427,0.732,6.06,...,0.311,7.9,0.354,1.2,0.381,2.4,0.402,1.0,0.341,0.411
2,200746,LaMarcus Aldridge,71,1.0,9.28,19.41,0.478,0.492,0.925,8.76,...,0.432,5.5,0.394,4.6,0.655,7.0,0.450,1.6,0.500,0.479
3,2546,Carmelo Anthony,39,1.0,8.97,19.41,0.462,0.502,0.768,7.44,...,0.430,5.5,0.366,3.2,0.511,5.4,0.509,3.3,0.500,0.480
4,2544,LeBron James,68,1.0,9.04,17.87,0.506,0.555,0.728,7.31,...,0.350,6.4,0.375,2.2,0.709,1.7,0.379,1.2,0.674,0.534


In [25]:
master_scoring.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'GP', 'FGA_FREQUENCY', 'FGM', 'FGA',
       'FG_PCT', 'EFG_PCT', 'FG2A_FREQUENCY', 'FG2M', 'FG2A', 'FG2_PCT',
       'FG3A_FREQUENCY', 'FG3M', 'FG3A', 'FG3_PCT', 'SEASON', 'OFF_RATING',
       'USG_PCT', 'PCT_PTS_FB', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT',
       'PCT_PTS_2PT_MR', 'DD2', 'TD3', 'TEAM_ABBREVIATION', 'MIN', 'DRIVE_PTS',
       'DRIVE_FG_PCT', 'CATCH_SHOOT_PTS', 'CATCH_SHOOT_FG_PCT', 'PULL_UP_PTS',
       'PULL_UP_FG_PCT', 'PAINT_TOUCH_PTS', 'PAINT_TOUCH_FG_PCT',
       'POST_TOUCH_PTS', 'POST_TOUCH_FG_PCT', 'ELBOW_TOUCH_PTS',
       'ELBOW_TOUCH_FG_PCT', 'EFF_FG_PCT'],
      dtype='object')

In [26]:
master_scoring = master_scoring.drop(['FGA_FREQUENCY', 'EFG_PCT', 'FG2A_FREQUENCY', 'FG3A_FREQUENCY', ], axis=1)

In [27]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    poss = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Possessions',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    poss['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(poss)

# concatenate all the dataframes into a single dataframe
poss = pd.concat(season_dfs)
poss.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'POINTS', 'TOUCHES', 'FRONT_CT_TOUCHES', 'TIME_OF_POSS',
       'AVG_SEC_PER_TOUCH', 'AVG_DRIB_PER_TOUCH', 'PTS_PER_TOUCH',
       'ELBOW_TOUCHES', 'POST_TOUCHES', 'PAINT_TOUCHES', 'PTS_PER_ELBOW_TOUCH',
       'PTS_PER_POST_TOUCH', 'PTS_PER_PAINT_TOUCH', 'SEASON'],
      dtype='object')

In [28]:
possession_df = poss[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'PTS_PER_ELBOW_TOUCH','PTS_PER_POST_TOUCH', 'PTS_PER_PAINT_TOUCH']]

In [29]:
master_scoring = pd.merge(master_scoring, possession_df, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='left')

In [30]:
master_scoring.shape

(4175, 39)

In [31]:
master_scoring.head()

,PLAYER_ID,PLAYER_NAME,GP,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,...,PAINT_TOUCH_PTS,PAINT_TOUCH_FG_PCT,POST_TOUCH_PTS,POST_TOUCH_FG_PCT,ELBOW_TOUCH_PTS,ELBOW_TOUCH_FG_PCT,EFF_FG_PCT,PTS_PER_ELBOW_TOUCH,PTS_PER_POST_TOUCH,PTS_PER_PAINT_TOUCH
0,201566,Russell Westbrook,67,9.33,20.60,0.453,8.06,16.33,0.494,1.27,...,1.5,0.487,0.8,0.463,0.7,0.621,0.455,0.725,0.598,0.676
1,977,Kobe Bryant,35,7.60,19.60,0.388,6.06,14.34,0.422,1.54,...,1.2,0.381,2.4,0.402,1.0,0.341,0.411,0.396,0.445,0.524
2,200746,LaMarcus Aldridge,71,9.28,19.41,0.478,8.76,17.94,0.488,0.52,...,4.6,0.655,7.0,0.450,1.6,0.500,0.479,0.403,0.559,1.015
3,2546,Carmelo Anthony,39,8.97,19.41,0.462,7.44,14.90,0.499,1.54,...,3.2,0.511,5.4,0.509,3.3,0.500,0.480,0.483,0.636,0.781
4,2544,LeBron James,68,9.04,17.87,0.506,7.31,13.00,0.562,1.74,...,2.2,0.709,1.7,0.379,1.2,0.674,0.534,0.638,0.459,0.981


# Create Master Passing/Playmaking

In [87]:
seasons = [f'{year}-{(year + 1) % 100:02}' for year in range(start, end + 1)]

start_hustle = 2018
if start < start_hustle:
    hustle_seasons = [f'{year}-{(year + 1) % 100:02}' for year in range(start_hustle, end + 1)]
else:
    hustle_seasons = seasons

print(seasons, hustle_seasons)

['2018-19', '2019-20', '2020-21', '2021-22', '2022-23'] ['2018-19', '2019-20', '2020-21', '2021-22', '2022-23']


In [33]:
# Define the desired parameters for the endpoint
per_mode = 'PerGame'
measure_type = 'Advanced'

# Retrieve the data using the API endpoint for each season
dfs = []
for season in seasons:
    stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season, per_mode_detailed=per_mode, measure_type_detailed_defense=measure_type)
    df = stats.get_data_frames()[0]
    df['SEASON'] = season
    dfs.append(df)

# Concatenate all the dataframes into one
gen_adv_stats_df = pd.concat(dfs, ignore_index=True)

# Print the dataframe
gen_adv_stats_df.head()



,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,E_PACE_RANK,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,SEASON
0,201985,AJ Price,AJ,1610612739,CLE,28.0,26,11,15,0.423,...,476,453,453,238,374,369,313,278,408,2014-15
1,201166,Aaron Brooks,Aaron,1610612741,CHI,30.0,82,50,32,0.610,...,371,388,388,226,82,74,125,112,279,2014-15
2,203932,Aaron Gordon,Aaron,1610612753,ORL,19.0,47,14,33,0.298,...,206,214,214,312,313,324,311,330,191,2014-15
3,203940,Adreian Payne,Adreian,1610612750,MIN,24.0,32,7,25,0.219,...,386,371,371,433,317,315,232,209,303,2014-15
4,201143,Al Horford,Al,1610612737,ATL,29.0,76,56,20,0.737,...,264,251,251,26,17,42,25,57,45,2014-15


In [34]:
gen_assist = gen_adv_stats_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'TEAM_ABBREVIATION', 'GP', 'AST_PCT', 'AST_TO', 'AST_RATIO']]
gen_assist.head()

,PLAYER_ID,PLAYER_NAME,SEASON,TEAM_ABBREVIATION,GP,AST_PCT,AST_TO,AST_RATIO
0,201985,AJ Price,2014-15,CLE,26,0.254,3.29,22.2
1,201166,Aaron Brooks,2014-15,CHI,82,0.245,1.66,20.0
2,203932,Aaron Gordon,2014-15,ORL,47,0.064,0.87,10.8
3,203940,Adreian Payne,2014-15,MIN,32,0.067,0.68,9.5
4,201143,Al Horford,2014-15,ATL,76,0.178,2.44,17.8


In [35]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    assist_eff = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Possessions',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    assist_eff['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(assist_eff)

# concatenate all the dataframes into a single dataframe
assist_effic = pd.concat(season_dfs)
assist_effic.head()


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,MIN,POINTS,TOUCHES,...,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,PTS_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,PTS_PER_ELBOW_TOUCH,PTS_PER_POST_TOUCH,PTS_PER_PAINT_TOUCH,SEASON
0,201985,AJ Price,1610612739,CLE,26,11,15,12.4,5.1,30.3,...,5.67,5.92,0.169,0.2,0.0,0.3,0.333,3.000,0.571,2014-15
1,201166,Aaron Brooks,1610612741,CHI,82,50,32,23.0,11.6,53.4,...,5.37,5.82,0.218,0.3,0.0,0.4,0.464,0.000,0.303,2014-15
2,203932,Aaron Gordon,1610612753,ORL,47,14,33,17.0,5.2,22.4,...,1.94,1.17,0.231,0.6,0.2,1.8,0.643,0.400,1.157,2014-15
3,203940,Adreian Payne,1610612750,MIN,32,7,25,23.1,6.7,39.3,...,1.73,0.44,0.169,3.3,1.9,3.3,0.327,0.344,0.648,2014-15
4,201143,Al Horford,1610612737,ATL,76,56,20,30.5,15.2,58.2,...,1.70,0.59,0.261,5.7,4.6,6.3,0.405,0.457,0.775,2014-15


In [36]:
assist_touches = assist_effic[['PLAYER_ID', 'PLAYER_NAME', 'TOUCHES', 'FRONT_CT_TOUCHES', 'TIME_OF_POSS', 'AVG_SEC_PER_TOUCH', 'AVG_DRIB_PER_TOUCH', 'ELBOW_TOUCHES', 'POST_TOUCHES', 'PAINT_TOUCHES', 'SEASON']]

In [37]:
assist_touches.head()

,PLAYER_ID,PLAYER_NAME,TOUCHES,FRONT_CT_TOUCHES,TIME_OF_POSS,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,SEASON
0,201985,AJ Price,30.3,11.6,2.9,5.67,5.92,0.2,0.0,0.3,2014-15
1,201166,Aaron Brooks,53.4,21.3,4.8,5.37,5.82,0.3,0.0,0.4,2014-15
2,203932,Aaron Gordon,22.4,12.6,0.7,1.94,1.17,0.6,0.2,1.8,2014-15
3,203940,Adreian Payne,39.3,19.4,1.1,1.73,0.44,3.3,1.9,3.3,2014-15
4,201143,Al Horford,58.2,39.6,1.7,1.70,0.59,5.7,4.6,6.3,2014-15


In [38]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    passing = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Passing',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    passing['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(passing)

# concatenate all the dataframes into a single dataframe
pass_df = pd.concat(season_dfs)
pass_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,MIN,PASSES_MADE,PASSES_RECEIVED,AST,FT_AST,SECONDARY_AST,POTENTIAL_AST,AST_POINTS_CREATED,AST_ADJ,AST_TO_PASS_PCT,AST_TO_PASS_PCT_ADJ,SEASON
0,201985,AJ Price,1610612739,CLE,26,11,15,12.4,22.8,26.9,1.8,0.2,0.2,4.3,4.4,2.2,0.078,0.098,2014-15
1,201166,Aaron Brooks,1610612741,CHI,82,50,32,23.0,38.7,47.7,3.2,0.5,0.6,7.5,8.2,4.2,0.082,0.109,2014-15
2,203932,Aaron Gordon,1610612753,ORL,47,14,33,17.0,16.3,11.8,0.7,0.0,0.1,1.6,1.8,0.9,0.043,0.055,2014-15
3,203940,Adreian Payne,1610612750,MIN,32,7,25,23.1,29.6,18.9,0.9,0.1,0.0,2.6,2.4,1.1,0.032,0.038,2014-15
4,201143,Al Horford,1610612737,ATL,76,56,20,30.5,42.4,39.5,3.2,0.2,0.4,5.5,8.3,3.8,0.076,0.089,2014-15


In [39]:
pass_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'PASSES_MADE', 'PASSES_RECEIVED', 'AST', 'FT_AST',
       'SECONDARY_AST', 'POTENTIAL_AST', 'AST_POINTS_CREATED', 'AST_ADJ',
       'AST_TO_PASS_PCT', 'AST_TO_PASS_PCT_ADJ', 'SEASON'],
      dtype='object')

In [40]:
passing_df = pass_df[['PLAYER_ID', 'PLAYER_NAME', 'AST', 'FT_AST','SECONDARY_AST', 'POTENTIAL_AST', 'AST_POINTS_CREATED', 'AST_TO_PASS_PCT', 'SEASON']]
passing_df.head()

,PLAYER_ID,PLAYER_NAME,AST,FT_AST,SECONDARY_AST,POTENTIAL_AST,AST_POINTS_CREATED,AST_TO_PASS_PCT,SEASON
0,201985,AJ Price,1.8,0.2,0.2,4.3,4.4,0.078,2014-15
1,201166,Aaron Brooks,3.2,0.5,0.6,7.5,8.2,0.082,2014-15
2,203932,Aaron Gordon,0.7,0.0,0.1,1.6,1.8,0.043,2014-15
3,203940,Adreian Payne,0.9,0.1,0.0,2.6,2.4,0.032,2014-15
4,201143,Al Horford,3.2,0.2,0.4,5.5,8.3,0.076,2014-15


In [41]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    drives = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Drives',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    drives['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(drives)

# concatenate all the dataframes into a single dataframe
drives_df = pd.concat(season_dfs)
drives_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,MIN,DRIVES,DRIVE_FGM,...,DRIVE_PTS_PCT,DRIVE_PASSES,DRIVE_PASSES_PCT,DRIVE_AST,DRIVE_AST_PCT,DRIVE_TOV,DRIVE_TOV_PCT,DRIVE_PF,DRIVE_PF_PCT,SEASON
0,201985,AJ Price,1610612739,CLE,26,11,15,12.4,3.9,0.7,...,0.416,1.7,0.436,0.3,0.089,0.2,0.059,0.2,0.040,2014-15
1,201166,Aaron Brooks,1610612741,CHI,82,50,32,23.0,10.4,2.0,...,0.468,3.8,0.362,0.9,0.089,0.7,0.069,0.5,0.047,2014-15
2,203932,Aaron Gordon,1610612753,ORL,47,14,33,17.0,2.0,0.5,...,0.598,0.4,0.217,0.0,0.022,0.2,0.120,0.2,0.087,2014-15
3,203940,Adreian Payne,1610612750,MIN,32,7,25,23.1,1.8,0.4,...,0.589,0.2,0.125,0.0,0.018,0.2,0.107,0.2,0.089,2014-15
4,201143,Al Horford,1610612737,ATL,76,56,20,30.5,1.9,0.5,...,0.685,0.6,0.315,0.2,0.084,0.1,0.028,0.2,0.091,2014-15


In [42]:
drives_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'DRIVES', 'DRIVE_FGM', 'DRIVE_FGA', 'DRIVE_FG_PCT',
       'DRIVE_FTM', 'DRIVE_FTA', 'DRIVE_FT_PCT', 'DRIVE_PTS', 'DRIVE_PTS_PCT',
       'DRIVE_PASSES', 'DRIVE_PASSES_PCT', 'DRIVE_AST', 'DRIVE_AST_PCT',
       'DRIVE_TOV', 'DRIVE_TOV_PCT', 'DRIVE_PF', 'DRIVE_PF_PCT', 'SEASON'],
      dtype='object')

In [43]:
drives_df = drives_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'DRIVE_PASSES', 'DRIVE_AST', 'DRIVE_TOV']]

In [44]:
drives_df.head()

,PLAYER_ID,PLAYER_NAME,SEASON,DRIVE_PASSES,DRIVE_AST,DRIVE_TOV
0,201985,AJ Price,2014-15,1.7,0.3,0.2
1,201166,Aaron Brooks,2014-15,3.8,0.9,0.7
2,203932,Aaron Gordon,2014-15,0.4,0.0,0.2
3,203940,Adreian Payne,2014-15,0.2,0.0,0.2
4,201143,Al Horford,2014-15,0.6,0.2,0.1


In [45]:
# import necessary packages
from nba_api.stats.endpoints import leaguedashptstats

# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    elbow = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='ElbowTouch',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    elbow['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(elbow)

# concatenate all the dataframes into a single dataframe
elbow_df = pd.concat(season_dfs)
elbow_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'TOUCHES', 'ELBOW_TOUCHES', 'ELBOW_TOUCH_FGM',
       'ELBOW_TOUCH_FGA', 'ELBOW_TOUCH_FG_PCT', 'ELBOW_TOUCH_FTM',
       'ELBOW_TOUCH_FTA', 'ELBOW_TOUCH_FT_PCT', 'ELBOW_TOUCH_PTS',
       'ELBOW_TOUCH_PASSES', 'ELBOW_TOUCH_AST', 'ELBOW_TOUCH_AST_PCT',
       'ELBOW_TOUCH_TOV', 'ELBOW_TOUCH_TOV_PCT', 'ELBOW_TOUCH_FOULS',
       'ELBOW_TOUCH_PASSES_PCT', 'ELBOW_TOUCH_FOULS_PCT',
       'ELBOW_TOUCH_PTS_PCT', 'SEASON'],
      dtype='object')

In [46]:
elbow_df = elbow_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'ELBOW_TOUCH_PASSES', 'ELBOW_TOUCH_AST', 'ELBOW_TOUCH_TOV']]
elbow_df.head()

,PLAYER_ID,PLAYER_NAME,SEASON,ELBOW_TOUCH_PASSES,ELBOW_TOUCH_AST,ELBOW_TOUCH_TOV
0,201985,AJ Price,2014-15,0.1,0.0,0.0
1,201166,Aaron Brooks,2014-15,0.1,0.0,0.0
2,203932,Aaron Gordon,2014-15,0.3,0.0,0.0
3,203940,Adreian Payne,2014-15,1.5,0.2,0.3
4,201143,Al Horford,2014-15,3.2,0.4,0.1


In [49]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    post = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='PostTouch',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    post['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(post)

# concatenate all the dataframes into a single dataframe
post_df = pd.concat(season_dfs)
post_df = post_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'POST_TOUCH_PASSES', 'POST_TOUCH_AST', 'POST_TOUCH_TOV']]
post_df.head()

,PLAYER_ID,PLAYER_NAME,SEASON,POST_TOUCH_PASSES,POST_TOUCH_AST,POST_TOUCH_TOV
0,201985,AJ Price,2014-15,0.0,0.0,0.0
1,201166,Aaron Brooks,2014-15,0.0,0.0,0.0
2,203932,Aaron Gordon,2014-15,0.0,0.0,0.0
3,203940,Adreian Payne,2014-15,0.6,0.1,0.1
4,201143,Al Horford,2014-15,1.5,0.4,0.3


In [50]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    paint = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='PaintTouch',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    paint['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(paint)

# concatenate all the dataframes into a single dataframe
paint_df = pd.concat(season_dfs)
paint_df = paint_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'PAINT_TOUCH_PASSES', 'PAINT_TOUCH_AST', 'PAINT_TOUCH_TOV']]
paint_df.head()

,PLAYER_ID,PLAYER_NAME,SEASON,PAINT_TOUCH_PASSES,PAINT_TOUCH_AST,PAINT_TOUCH_TOV
0,201985,AJ Price,2014-15,0.1,0.0,0.1
1,201166,Aaron Brooks,2014-15,0.2,0.0,0.0
2,203932,Aaron Gordon,2014-15,0.3,0.0,0.0
3,203940,Adreian Payne,2014-15,1.2,0.2,0.1
4,201143,Al Horford,2014-15,2.5,0.5,0.1


In [88]:
from nba_api.stats.endpoints import LeagueHustleStatsPlayer

# Create an empty list to store the player tracking data
hustle_df_list = []

# Loop over the seasons and retrieve the data for each season
for season in hustle_seasons:
    # Make the API call to retrieve the player tracking data
    hustle_stats = LeagueHustleStatsPlayer(per_mode_time = 'PerGame', season_type_all_star = 'Regular Season', season=season)
    hustle_df = hustle_stats.get_data_frames()[0]
    
    # Add a season column to the dataframe
    hustle_df['SEASON'] = season
    
    # Append the dataframe to the list
    hustle_df_list.append(hustle_df)

# Concatenate the list of dataframes into a single dataframe
hustle_df = pd.concat(hustle_df_list, ignore_index=True)

# Display the dataframe
hustle_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,G,MIN,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,...,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,PCT_BOX_OUTS_OFF,PCT_BOX_OUTS_DEF,PCT_BOX_OUTS_TEAM_REB,PCT_BOX_OUTS_REB,SEASON
0,203932,Aaron Gordon,1610612753,ORL,23.0,78,33.8,6.53,4.42,2.10,...,0.22,2.18,2.40,1.14,0.38,0.091,0.909,0.856,0.288,2018-19
1,1628988,Aaron Holiday,1610612754,IND,22.0,50,12.9,2.50,1.46,1.04,...,0.00,0.46,0.46,0.26,0.00,0.000,1.000,0.929,0.000,2018-19
2,1627846,Abdel Nader,1610612760,OKC,25.0,61,11.4,3.77,2.15,1.62,...,0.00,0.79,0.79,0.44,0.16,0.000,1.000,0.794,0.294,2018-19
3,201143,Al Horford,1610612738,BOS,33.0,68,29.0,11.68,8.79,2.88,...,0.49,6.04,6.53,3.25,1.13,0.074,0.926,0.860,0.300,2018-19
4,202329,Al-Farouq Aminu,1610612757,POR,28.0,81,28.3,7.42,4.37,3.05,...,0.28,3.27,3.56,1.64,0.70,0.080,0.920,0.875,0.375,2018-19


In [89]:
hustle_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'G',
       'MIN', 'CONTESTED_SHOTS', 'CONTESTED_SHOTS_2PT', 'CONTESTED_SHOTS_3PT',
       'DEFLECTIONS', 'CHARGES_DRAWN', 'SCREEN_ASSISTS', 'SCREEN_AST_PTS',
       'OFF_LOOSE_BALLS_RECOVERED', 'DEF_LOOSE_BALLS_RECOVERED',
       'LOOSE_BALLS_RECOVERED', 'PCT_LOOSE_BALLS_RECOVERED_OFF',
       'PCT_LOOSE_BALLS_RECOVERED_DEF', 'OFF_BOXOUTS', 'DEF_BOXOUTS',
       'BOX_OUTS', 'BOX_OUT_PLAYER_TEAM_REBS', 'BOX_OUT_PLAYER_REBS',
       'PCT_BOX_OUTS_OFF', 'PCT_BOX_OUTS_DEF', 'PCT_BOX_OUTS_TEAM_REB',
       'PCT_BOX_OUTS_REB', 'SEASON'],
      dtype='object')

In [90]:
hustle_assist = hustle_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'SCREEN_ASSISTS']]

In [91]:
hustle_assist.head()

,PLAYER_ID,PLAYER_NAME,SEASON,SCREEN_ASSISTS
0,203932,Aaron Gordon,2018-19,0.87
1,1628988,Aaron Holiday,2018-19,0.02
2,1627846,Abdel Nader,2018-19,0.02
3,201143,Al Horford,2018-19,3.32
4,202329,Al-Farouq Aminu,2018-19,0.96


In [92]:
# Merge the hustle assist and paint dataframes
merge1 = pd.merge(hustle_assist, paint_df, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')

# Merge the post, elbow, and drives dataframes
merge2 = pd.merge(post_df, elbow_df, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')
merge3 = pd.merge(merge2, drives_df, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')

# Merge the passing, assist touches, and general assist dataframes
merge4 = pd.merge(passing_df, assist_touches, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')
merge5 = pd.merge(merge4, gen_assist, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')

# Merge all the merged dataframes
master_assist = pd.merge(merge1, merge5, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')


In [93]:
master_assist.head()

,PLAYER_ID,PLAYER_NAME,SEASON,SCREEN_ASSISTS,PAINT_TOUCH_PASSES,PAINT_TOUCH_AST,PAINT_TOUCH_TOV,AST,FT_AST,SECONDARY_AST,...,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,TEAM_ABBREVIATION,GP,AST_PCT,AST_TO,AST_RATIO
0,203932,Aaron Gordon,2018-19,0.87,1.1,0.2,0.2,3.7,0.1,0.4,...,2.62,1.76,2.6,2.9,4.3,ORL,78,0.166,1.78,18.0
1,1628988,Aaron Holiday,2018-19,0.02,0.1,0.0,0.0,1.7,0.1,0.3,...,4.09,3.82,0.2,0.0,0.1,IND,50,0.180,2.17,21.2
2,1627846,Abdel Nader,2018-19,0.02,0.0,0.0,0.0,0.3,0.0,0.1,...,2.18,1.53,0.1,0.0,0.3,OKC,61,0.044,0.77,7.3
3,201143,Al Horford,2018-19,3.32,1.9,0.5,0.1,4.2,0.1,0.5,...,1.78,0.76,4.1,3.7,5.2,BOS,68,0.203,2.77,24.7
4,202329,Al-Farouq Aminu,2018-19,0.96,0.7,0.2,0.1,1.3,0.1,0.2,...,1.91,0.92,1.1,0.3,2.4,POR,81,0.057,1.44,12.3


# Master Rebounding

In [94]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    rebound = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Rebounding',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    rebound['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(rebound)

# concatenate all the dataframes into a single dataframe
rebound_df = pd.concat(season_dfs)
rebound_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'OREB', 'OREB_CONTEST', 'OREB_UNCONTEST',
       'OREB_CONTEST_PCT', 'OREB_CHANCES', 'OREB_CHANCE_PCT',
       'OREB_CHANCE_DEFER', 'OREB_CHANCE_PCT_ADJ', 'AVG_OREB_DIST', 'DREB',
       'DREB_CONTEST', 'DREB_UNCONTEST', 'DREB_CONTEST_PCT', 'DREB_CHANCES',
       'DREB_CHANCE_PCT', 'DREB_CHANCE_DEFER', 'DREB_CHANCE_PCT_ADJ',
       'AVG_DREB_DIST', 'REB', 'REB_CONTEST', 'REB_UNCONTEST',
       'REB_CONTEST_PCT', 'REB_CHANCES', 'REB_CHANCE_PCT', 'REB_CHANCE_DEFER',
       'REB_CHANCE_PCT_ADJ', 'AVG_REB_DIST', 'SEASON'],
      dtype='object')

In [95]:
rebound_tracking = rebound_df[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'OREB', 'OREB_CONTEST', 'OREB_UNCONTEST', 'OREB_CHANCES', 'AVG_OREB_DIST', 'OREB_CHANCE_DEFER', 'DREB', 'DREB_CONTEST', 'DREB_UNCONTEST', 'DREB_CHANCES', 'DREB_CHANCE_DEFER', 'AVG_DREB_DIST', 'SEASON']]
rebound_tracking.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,OREB,OREB_CONTEST,OREB_UNCONTEST,OREB_CHANCES,AVG_OREB_DIST,OREB_CHANCE_DEFER,DREB,DREB_CONTEST,DREB_UNCONTEST,DREB_CHANCES,DREB_CHANCE_DEFER,AVG_DREB_DIST,SEASON
0,203932,Aaron Gordon,ORL,76,33.8,1.7,1.0,0.7,3.5,7.2,0.1,5.8,1.3,4.3,8.2,0.6,6.3,2018-19
1,1628988,Aaron Holiday,IND,28,17.3,0.2,0.1,0.1,0.6,16.0,0.0,2.2,0.1,2.1,3.5,0.3,9.0,2018-19
2,1627846,Abdel Nader,OKC,45,13.7,0.3,0.0,0.3,1.0,16.8,0.0,2.2,0.4,1.8,3.5,0.2,7.3,2018-19
3,201143,Al Horford,BOS,68,29.0,1.8,0.9,0.9,3.9,8.2,0.1,5.0,1.6,3.3,8.9,0.8,5.8,2018-19
4,202329,Al-Farouq Aminu,POR,81,28.3,1.4,0.8,0.6,3.6,7.0,0.1,6.1,1.5,4.6,9.4,0.7,5.2,2018-19


In [96]:
rebound_hustle = hustle_df[['PLAYER_ID', 'PLAYER_NAME', 'OFF_BOXOUTS', 'DEF_BOXOUTS', 'BOX_OUT_PLAYER_REBS', 'SEASON']]

In [97]:
rebound_hustle.head()

,PLAYER_ID,PLAYER_NAME,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUT_PLAYER_REBS,SEASON
0,203932,Aaron Gordon,0.22,2.18,0.38,2018-19
1,1628988,Aaron Holiday,0.00,0.46,0.00,2018-19
2,1627846,Abdel Nader,0.00,0.79,0.16,2018-19
3,201143,Al Horford,0.49,6.04,1.13,2018-19
4,202329,Al-Farouq Aminu,0.28,3.27,0.70,2018-19


In [98]:
master_rebound = pd.merge(rebound_hustle, rebound_tracking, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how='outer')


In [99]:
master_rebound.head()

,PLAYER_ID,PLAYER_NAME,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUT_PLAYER_REBS,SEASON,TEAM_ABBREVIATION,GP,MIN,OREB,...,OREB_UNCONTEST,OREB_CHANCES,AVG_OREB_DIST,OREB_CHANCE_DEFER,DREB,DREB_CONTEST,DREB_UNCONTEST,DREB_CHANCES,DREB_CHANCE_DEFER,AVG_DREB_DIST
0,203932,Aaron Gordon,0.22,2.18,0.38,2018-19,ORL,76.0,33.8,1.7,...,0.7,3.5,7.2,0.1,5.8,1.3,4.3,8.2,0.6,6.3
1,1628988,Aaron Holiday,0.00,0.46,0.00,2018-19,IND,28.0,17.3,0.2,...,0.1,0.6,16.0,0.0,2.2,0.1,2.1,3.5,0.3,9.0
2,1627846,Abdel Nader,0.00,0.79,0.16,2018-19,OKC,45.0,13.7,0.3,...,0.3,1.0,16.8,0.0,2.2,0.4,1.8,3.5,0.2,7.3
3,201143,Al Horford,0.49,6.04,1.13,2018-19,BOS,68.0,29.0,1.8,...,0.9,3.9,8.2,0.1,5.0,1.6,3.3,8.9,0.8,5.8
4,202329,Al-Farouq Aminu,0.28,3.27,0.70,2018-19,POR,81.0,28.3,1.4,...,0.6,3.6,7.0,0.1,6.1,1.5,4.6,9.4,0.7,5.2


# Master Defence

In [100]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    defense = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Defense',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    defense['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(defense)

# concatenate all the dataframes into a single dataframe
defense_df = pd.concat(season_dfs)
defense_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'STL', 'BLK', 'DREB', 'DEF_RIM_FGM', 'DEF_RIM_FGA',
       'DEF_RIM_FG_PCT', 'SEASON'],
      dtype='object')

In [101]:
defensive_tracking = defense_df[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'SEASON', 'GP', 'MIN', 'STL', 'BLK', 'DEF_RIM_FGA', 'DEF_RIM_FGM', 'DEF_RIM_FG_PCT']]
defensive_tracking.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,GP,MIN,STL,BLK,DEF_RIM_FGA,DEF_RIM_FGM,DEF_RIM_FG_PCT
0,203932,Aaron Gordon,ORL,2018-19,76,33.8,0.7,0.7,3.0,1.9,0.642
1,1628988,Aaron Holiday,IND,2018-19,50,12.9,0.4,0.3,1.0,0.6,0.627
2,1627846,Abdel Nader,OKC,2018-19,60,11.4,0.3,0.2,1.3,1.0,0.734
3,201143,Al Horford,BOS,2018-19,68,29.0,0.9,1.3,5.5,3.3,0.597
4,202329,Al-Farouq Aminu,POR,2018-19,81,28.3,0.8,0.4,3.2,2.1,0.661


In [102]:
hustle_def = hustle_df[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'CONTESTED_SHOTS_2PT', 'CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 'CHARGES_DRAWN', 'LOOSE_BALLS_RECOVERED']]
hustle_def.head()

,PLAYER_ID,PLAYER_NAME,SEASON,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,DEFLECTIONS,CHARGES_DRAWN,LOOSE_BALLS_RECOVERED
0,203932,Aaron Gordon,2018-19,4.42,2.10,1.29,0.01,1.22
1,1628988,Aaron Holiday,2018-19,1.46,1.04,0.66,0.00,0.34
2,1627846,Abdel Nader,2018-19,2.15,1.62,0.51,0.02,0.41
3,201143,Al Horford,2018-19,8.79,2.88,1.38,0.00,0.84
4,202329,Al-Farouq Aminu,2018-19,4.37,3.05,1.91,0.02,0.99


In [103]:
 # create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player scoring data for the season
    scoring_data = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
        measure_type_detailed_defense='Defense',
        per_mode_detailed = 'PerGame'
    ).get_data_frames()[0]
    scoring_data['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(scoring_data)

# concatenate all the dataframes into a single dataframe
gen = pd.concat(season_dfs)
gen.head()   

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,STL_RANK,PCT_STL_RANK,BLK_RANK,PCT_BLK_RANK,OPP_PTS_OFF_TOV_RANK,OPP_PTS_2ND_CHANCE_RANK,OPP_PTS_FB_RANK,OPP_PTS_PAINT_RANK,DEF_WS_RANK,SEASON
0,203932,Aaron Gordon,Aaron,1610612753,ORL,23.0,78,40,38,0.513,...,172,347,80,201,456,404,426,494,21,2018-19
1,1628988,Aaron Holiday,Aaron,1610612754,IND,22.0,50,31,19,0.620,...,331,181,278,209,117,148,139,131,174,2018-19
2,1627846,Abdel Nader,Abdel,1610612760,OKC,25.0,61,38,23,0.623,...,398,351,330,251,104,143,128,120,341,2018-19
3,201143,Al Horford,Al,1610612738,BOS,33.0,68,41,27,0.603,...,122,314,25,73,351,462,418,411,57,2018-19
4,202329,Al-Farouq Aminu,Al-Farouq,1610612757,POR,28.0,81,52,29,0.642,...,137,256,182,295,343,425,411,420,104,2018-19


In [104]:
gen.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'DEF_RATING', 'DREB', 'DREB_PCT',
       'PCT_DREB', 'STL', 'PCT_STL', 'BLK', 'PCT_BLK', 'OPP_PTS_OFF_TOV',
       'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT', 'DEF_WS',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'DEF_RATING_RANK', 'DREB_RANK', 'DREB_PCT_RANK', 'PCT_DREB_RANK',
       'STL_RANK', 'PCT_STL_RANK', 'BLK_RANK', 'PCT_BLK_RANK',
       'OPP_PTS_OFF_TOV_RANK', 'OPP_PTS_2ND_CHANCE_RANK', 'OPP_PTS_FB_RANK',
       'OPP_PTS_PAINT_RANK', 'DEF_WS_RANK', 'SEASON'],
      dtype='object')

In [105]:
opp_scoring = gen[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT']]
opp_scoring.head()

,PLAYER_ID,PLAYER_NAME,SEASON,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT
0,203932,Aaron Gordon,2018-19,10.4,7.4,8.1,33.5
1,1628988,Aaron Holiday,2018-19,3.9,3.6,3.5,12.0
2,1627846,Abdel Nader,2018-19,3.7,3.5,3.3,11.4
3,201143,Al Horford,2018-19,8.2,8.5,8.0,27.8
4,202329,Al-Farouq Aminu,2018-19,8.2,7.9,7.9,28.2


In [123]:
master_def = defensive_tracking.merge(opp_scoring, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how = 'outer')
master_def = master_def.merge(hustle_def, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'], how = 'outer')
master_def.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,GP,MIN,STL,BLK,DEF_RIM_FGA,DEF_RIM_FGM,DEF_RIM_FG_PCT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,DEFLECTIONS,CHARGES_DRAWN,LOOSE_BALLS_RECOVERED
0,203932,Aaron Gordon,ORL,2018-19,76.0,33.8,0.7,0.7,3.0,1.9,0.642,10.4,7.4,8.1,33.5,4.42,2.10,1.29,0.01,1.22
1,1628988,Aaron Holiday,IND,2018-19,50.0,12.9,0.4,0.3,1.0,0.6,0.627,3.9,3.6,3.5,12.0,1.46,1.04,0.66,0.00,0.34
2,1627846,Abdel Nader,OKC,2018-19,60.0,11.4,0.3,0.2,1.3,1.0,0.734,3.7,3.5,3.3,11.4,2.15,1.62,0.51,0.02,0.41
3,201143,Al Horford,BOS,2018-19,68.0,29.0,0.9,1.3,5.5,3.3,0.597,8.2,8.5,8.0,27.8,8.79,2.88,1.38,0.00,0.84
4,202329,Al-Farouq Aminu,POR,2018-19,81.0,28.3,0.8,0.4,3.2,2.1,0.661,8.2,7.9,7.9,28.2,4.37,3.05,1.91,0.02,0.99


# Hustle

In [107]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    speed = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='SpeedDistance',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    speed['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(speed)

# concatenate all the dataframes into a single dataframe
speed_df = pd.concat(season_dfs)
speed_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'MIN1', 'DIST_FEET', 'DIST_MILES', 'DIST_MILES_OFF',
       'DIST_MILES_DEF', 'AVG_SPEED', 'AVG_SPEED_OFF', 'AVG_SPEED_DEF',
       'SEASON'],
      dtype='object')

In [108]:
speed_dist = speed_df[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP','MIN', 'DIST_MILES','AVG_SPEED', 'SEASON']]

In [110]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player tracking shooting efficiency data for the season
    drive_pf = leaguedashptstats.LeagueDashPtStats(
        season=season_str,
        pt_measure_type='Drives',
        player_or_team='Player',
        per_mode_simple = 'PerGame'
    ).get_data_frames()[0]
    drive_pf['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(drive_pf)

# concatenate all the dataframes into a single dataframe
drive_pf_df = pd.concat(season_dfs)
drive_pf_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W',
       'L', 'MIN', 'DRIVES', 'DRIVE_FGM', 'DRIVE_FGA', 'DRIVE_FG_PCT',
       'DRIVE_FTM', 'DRIVE_FTA', 'DRIVE_FT_PCT', 'DRIVE_PTS', 'DRIVE_PTS_PCT',
       'DRIVE_PASSES', 'DRIVE_PASSES_PCT', 'DRIVE_AST', 'DRIVE_AST_PCT',
       'DRIVE_TOV', 'DRIVE_TOV_PCT', 'DRIVE_PF', 'DRIVE_PF_PCT', 'SEASON'],
      dtype='object')

In [111]:
drive_fouls = drive_pf_df[['PLAYER_ID', 'PLAYER_NAME', 'DRIVE_PF', 'SEASON']]

In [112]:
drive_fouls.head()

,PLAYER_ID,PLAYER_NAME,DRIVE_PF,SEASON
0,203932,Aaron Gordon,0.5,2018-19
1,1628988,Aaron Holiday,0.1,2018-19
2,1627846,Abdel Nader,0.1,2018-19
3,201143,Al Horford,0.1,2018-19
4,202329,Al-Farouq Aminu,0.2,2018-19


In [113]:
speed_dist.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,DIST_MILES,AVG_SPEED,SEASON
0,203932,Aaron Gordon,ORL,76,33.78,2.56,4.28,2018-19
1,1628988,Aaron Holiday,IND,50,12.92,1.01,4.35,2018-19
2,1627846,Abdel Nader,OKC,60,11.43,0.92,4.47,2018-19
3,201143,Al Horford,BOS,68,29.01,2.04,3.98,2018-19
4,202329,Al-Farouq Aminu,POR,81,28.29,2.14,4.25,2018-19


In [114]:
from nba_api.stats.endpoints import leaguedashplayerstats
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player scoring data for the season
    misc_data = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
        per_mode_detailed = 'PerGame', 
        season_type_all_star = 'Regular Season'
    ).get_data_frames()[0]
    misc_data['SEASON'] = season_str  # add new column for season
    # append the dataframe to the list
    season_dfs.append(misc_data)

# concatenate all the dataframes into a single dataframe
misc_data = pd.concat(season_dfs)
pf_drawn = misc_data[['PLAYER_ID', 'PLAYER_NAME', 'PFD', 'SEASON']]

In [115]:
pf_drawn.head()

,PLAYER_ID,PLAYER_NAME,PFD,SEASON
0,203932,Aaron Gordon,3.4,2018-19
1,1628988,Aaron Holiday,1.2,2018-19
2,1627846,Abdel Nader,0.6,2018-19
3,201143,Al Horford,1.8,2018-19
4,202329,Al-Farouq Aminu,1.8,2018-19


In [116]:
master_hustle = pf_drawn.merge(speed_dist, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON']).merge(drive_fouls, on=['PLAYER_ID', 'PLAYER_NAME', 'SEASON'])


In [117]:
master_hustle.head()

,PLAYER_ID,PLAYER_NAME,PFD,SEASON,TEAM_ABBREVIATION,GP,MIN,DIST_MILES,AVG_SPEED,DRIVE_PF
0,203932,Aaron Gordon,3.4,2018-19,ORL,76,33.78,2.56,4.28,0.5
1,1628988,Aaron Holiday,1.2,2018-19,IND,50,12.92,1.01,4.35,0.1
2,1627846,Abdel Nader,0.6,2018-19,OKC,60,11.43,0.92,4.47,0.1
3,201143,Al Horford,1.8,2018-19,BOS,68,29.01,2.04,3.98,0.1
4,202329,Al-Farouq Aminu,1.8,2018-19,POR,81,28.29,2.14,4.25,0.2


# Clutch

In [118]:
# create a list to hold the dataframes for each season
season_dfs = []

# loop through each season from 2014/15 to 2022/23
for season in range(start, end):
    season_str = str(season) + "-" + str(season+1)[-2:]  # create the season string
    # retrieve the player advanced stats data for the season
    adv_stats = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season_str,
        measure_type_detailed_defense='Advanced',
        per_mode_detailed = 'PerGame'
    ).get_data_frames()[0]
    # add season column
    adv_stats['SEASON'] = season_str
    # append the dataframe to the list
    season_dfs.append(adv_stats)

# concatenate all the dataframes into a single dataframe
gen_adv = pd.concat(season_dfs)
gen_adv.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,E_PACE_RANK,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,SEASON
0,203932,Aaron Gordon,Aaron,1610612753,ORL,23.0,78,40,38,0.513,...,454,466,466,132,51,46,61,57,229,2018-19
1,1628988,Aaron Holiday,Aaron,1610612754,IND,22.0,50,31,19,0.620,...,215,191,191,313,306,296,335,301,402,2018-19
2,1627846,Abdel Nader,Abdel,1610612760,OKC,25.0,61,38,23,0.623,...,175,104,104,384,329,325,408,396,316,2018-19
3,201143,Al Horford,Al,1610612738,BOS,33.0,68,41,27,0.603,...,338,373,373,59,76,101,79,111,68,2018-19
4,202329,Al-Farouq Aminu,Al-Farouq,1610612757,POR,28.0,81,52,29,0.642,...,252,335,335,193,152,141,209,199,281,2018-19


In [119]:
gen_adv.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RATING', 'OFF_RATING',
       'sp_work_OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'sp_work_DEF_RATING', 'E_NET_RATING', 'NET_RATING',
       'sp_work_NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT',
       'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'E_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'sp_work_PACE',
       'PIE', 'POSS', 'FGM', 'FGA', 'FGM_PG', 'FGA_PG', 'FG_PCT', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'E_OFF_RATING_RANK',
       'OFF_RATING_RANK', 'sp_work_OFF_RATING_RANK', 'E_DEF_RATING_RANK',
       'DEF_RATING_RANK', 'sp_work_DEF_RATING_RANK', 'E_NET_RATING_RANK',
       'NET_RATING_RANK', 'sp_work_NET_RATING_RANK', 'AST_PCT_RANK',
       'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK',
       'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'E_TOV_PCT_

In [120]:
master_clutch = gen_adv[['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'PIE', 'POSS', 'USG_PCT']]
master_clutch.head()

,PLAYER_ID,PLAYER_NAME,SEASON,TEAM_ABBREVIATION,GP,MIN,PIE,POSS,USG_PCT
0,203932,Aaron Gordon,2018-19,ORL,78,33.8,0.109,5456,0.213
1,1628988,Aaron Holiday,2018-19,IND,50,12.9,0.078,1393,0.206
2,1627846,Abdel Nader,2018-19,OKC,61,11.4,0.066,1512,0.148
3,201143,Al Horford,2018-19,BOS,68,29.0,0.134,4146,0.188
4,202329,Al-Farouq Aminu,2018-19,POR,81,28.3,0.097,4828,0.134


# Write to CSV

In [131]:
# Create a list of the dataframes
dfs = [master_scoring, master_assist, master_rebound, master_def, master_hustle, master_clutch]

# Loop through each dataframe and add the pidSzn column
for df in dfs:
    df.loc[:, 'pidSzn'] = df['PLAYER_ID'].astype(str) + '_' + df['SEASON']


master_scoring.to_csv('MasterScore.csv', index=False)
master_assist.to_csv('MasterPass.csv', index=False)
master_rebound.to_csv('MasterRebound.csv', index=False)
master_hustle.to_csv('MasterMisc.csv', index=False)
master_clutch.to_csv('MasterClutch.csv', index=False)
master_def.to_csv('MasterDefence.csv', index=False)